https://towardsdatascience.com/a-beginners-guide-to-sentiment-analysis-in-python-95e354ea84f6

In [5]:
import pandas as pd #работа с датафреймами- таблицами
import pymorphy2 as pm #части речи, формы
import re #регулярные выражения
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import time
from IPython.display import display 
import nltk
import numpy as np

In [19]:
df = pd.read_csv('../data/comments_filled_sent.csv')

In [20]:
df = df.loc[~df.normal_sentense.isna()]

In [36]:
df.shape

(198819, 18)

In [3]:
df = df[df['rating'] != 3]
df['sentiment'] = df['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [21]:
# df = df[df['score'] > 0]
df['sentiment'] = df['score'].apply(lambda rating : 1 if rating > 0 else -1)

In [22]:
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.8]
test = df[df['random_number'] > 0.8]

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['normal_sentense'])
test_matrix = vectorizer.transform(test['normal_sentense'])

In [24]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [25]:
X_train = train_matrix
X_test = test_matrix
y_train = train['sentiment']
y_test = test['sentiment']

In [26]:
lr.fit(X_train,y_train)

C:\Users\yupes\anaconda3\envs\python-gis\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [27]:
predictions = lr.predict(X_test)

In [28]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[11251,   698],
       [  689, 29483]], dtype=int64)

In [34]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

          -1       0.94      0.94      0.94     11949
           1       0.98      0.98      0.98     30172

    accuracy                           0.97     42121
   macro avg       0.96      0.96      0.96     42121
weighted avg       0.97      0.97      0.97     42121

